In [1]:
import pyodbc
import pandas as pd
import time

server = 'crypto-bot.database.windows.net'
database = 'binance'
username = 'cryptoAdmin'
password = 'Petit@Soleil'
driver = '{SQL Server}'

connector_str = 'DRIVER=' + driver + ';SERVER=tcp:' + server + ';PORT=1433;DATABASE=' + database + ';UID=' \
                + username + ';PWD=' + password

In [2]:
conn = pyodbc.connect(connector_str)

In [16]:
def extract_data(cryptoCode, limit=None, interval=None):
    if interval:
        if limit:
            query = """
                SELECT 
                TOP {} * 
                FROM future.MarketPrices mp
                LEFT JOIN future_ind.Ichimoku ich ON mp.MarketPriceId = ich.MarketPriceId
                LEFT JOIN future_ind.IchimokuSignal ich_sign ON mp.MarketPriceId = ich_sign.MarketPriceId
                WHERE mp.Symbol = '{}' AND mp.Interval = '{}'
            """.format(limit, cryptoCode, interval)
        else:
            query = """
                SELECT 
                    *
                FROM future.MarketPrices mp 
                LEFT JOIN future_ind.Ichimoku ich ON mp.MarketPriceId = ich.MarketPriceId
                LEFT JOIN future_ind.IchimokuSignal ich_sign ON mp.MarketPriceId = ich_sign.MarketPriceId
                WHERE mp.Symbol = '{}' AND mp.Interval = '{}'
            """.format(cryptoCode, interval)
    else:
        if limit:
            query = """
                SELECT 
                TOP {} * 
                FROM future.MarketPrices mp
                LEFT JOIN future_ind.Ichimoku ich ON mp.MarketPriceId = ich.MarketPriceId
                LEFT JOIN future_ind.IchimokuSignal ich_sign ON mp.MarketPriceId = ich_sign.MarketPriceId
                WHERE mp.Symbol = '{}'
            """.format(limit, cryptoCode)
        else:
            query = """
                SELECT 
                    *
                FROM future.MarketPrices mp 
                LEFT JOIN future_ind.Ichimoku ich ON mp.MarketPriceId = ich.MarketPriceId
                LEFT JOIN future_ind.IchimokuSignal ich_sign ON mp.MarketPriceId = ich_sign.MarketPriceId
                WHERE mp.Symbol = '{}'
            """.format(cryptoCode)
    return query



In [20]:
def generate_dataset(df, nb_timesteps):
    df_output = df.copy()
    # On ajoute une colonne à chaque itération
    for k in range(1,nb_timesteps+1):
        df_output[f"0P_T-{k}"] = df_output["OpenPrice"].shift(k)
    return df_output

In [21]:
query = extract_data('BTCBUSD')
df = pd.read_sql_query(query, conn)
df = df.drop('MarketPriceId', axis=1, errors='ignore').sort_values(by=['Interval', 'TimeOpenLong'])
df.columns

KeyboardInterrupt: 

In [5]:
df.head()

,Symbol,Interval,TimeOpenLong,TimeCloseLong,DateOpen,DateClose,OpenPrice,ClosePrice,MaxPrice,MinPrice,...,EcratTenkenKinjun,TendancePrice,TendanceLaginSpan,MarketPriceId_laginSpan,Reference,Variable,SignalType,LevelType,SignalSens,Comment
63,BTCBUSD,15m,1610434800000,1.610436e+12,2021-01-12 07:00:00.0000000,2021-01-12 07:14:59.9990000,36060.5,35723.4,36060.5,35715.6,...,NaN,NA,NA,NaN,None,None,None,None,None,None
64,BTCBUSD,15m,1610435700000,1.610437e+12,2021-01-12 07:15:00.0000000,2021-01-12 07:29:59.9990000,35735.1,35794.4,35883.6,35714.0,...,NaN,NA,NA,NaN,None,None,None,None,None,None
65,BTCBUSD,15m,1610436600000,1.610437e+12,2021-01-12 07:30:00.0000000,2021-01-12 07:44:59.9990000,35837.0,36590.3,36615.1,35837.0,...,NaN,NA,NA,NaN,None,None,None,None,None,None
66,BTCBUSD,15m,1610437500000,1.610438e+12,2021-01-12 07:45:00.0000000,2021-01-12 07:59:59.9990000,36523.4,36433.4,36535.6,36372.8,...,NaN,NA,NA,NaN,None,None,None,None,None,None
67,BTCBUSD,15m,1610438400000,1.610439e+12,2021-01-12 08:00:00.0000000,2021-01-12 08:14:59.9990000,36453.3,36108.3,36496.1,36015.4,...,NaN,NA,NA,NaN,None,None,None,None,None,None


In [6]:
df_1d = df[df["Interval"] == '1d'].sort_values(by=['TimeOpenLong']).reset_index().drop('index', axis=1)
df_1d.head(10)

,Symbol,Interval,TimeOpenLong,TimeCloseLong,DateOpen,DateClose,OpenPrice,ClosePrice,MaxPrice,MinPrice,...,EcratTenkenKinjun,TendancePrice,TendanceLaginSpan,MarketPriceId_laginSpan,Reference,Variable,SignalType,LevelType,SignalSens,Comment
0,BTCBUSD,1d,1610409600000,1.610496e+12,2021-01-12 00:00:00.0000000,2021-01-12 23:59:59.9990000,36060.5,34021.1,36615.1,32520.4,...,NaN,NA,NA,NaN,None,None,None,None,None,None
1,BTCBUSD,1d,1610496000000,1.610582e+12,2021-01-13 00:00:00.0000000,2021-01-13 23:59:59.9990000,33877.8,37355.7,37763.5,32346.7,...,NaN,NA,NA,NaN,None,None,None,None,None,None
2,BTCBUSD,1d,1610582400000,1.610669e+12,2021-01-14 00:00:00.0000000,2021-01-14 23:59:59.9990000,37451.4,39091.6,40132.9,36615.4,...,NaN,NA,NA,NaN,None,None,None,None,None,None
3,BTCBUSD,1d,1610668800000,1.610755e+12,2021-01-15 00:00:00.0000000,2021-01-15 23:59:59.9990000,39164.4,36737.6,39703.3,34362.0,...,NaN,NA,NA,NaN,None,None,None,None,None,None
4,BTCBUSD,1d,1610755200000,1.610842e+12,2021-01-16 00:00:00.0000000,2021-01-16 23:59:59.9990000,36739.0,35975.0,38149.1,35299.7,...,NaN,NA,NA,NaN,None,None,None,None,None,None
5,BTCBUSD,1d,1610841600000,1.610928e+12,2021-01-17 00:00:00.0000000,2021-01-17 23:59:59.9990000,35990.6,35818.3,36858.7,33823.5,...,NaN,NA,NA,NaN,None,None,None,None,None,None
6,BTCBUSD,1d,1610928000000,1.611014e+12,2021-01-18 00:00:00.0000000,2021-01-18 23:59:59.9990000,35830.4,36632.1,37510.5,34704.7,...,NaN,NA,NA,NaN,None,None,None,None,None,None
7,BTCBUSD,1d,1611014400000,1.611101e+12,2021-01-19 00:00:00.0000000,2021-01-19 23:59:59.9990000,36660.0,35839.9,37874.8,35839.9,...,NaN,NA,NA,NaN,None,None,None,None,None,None
8,BTCBUSD,1d,1611100800000,1.611187e+12,2021-01-20 00:00:00.0000000,2021-01-20 23:59:59.9990000,35839.9,35514.4,36410.1,33358.1,...,NaN,NA,NA,NaN,Tenken,Prix,Test,Ressistance,Sell,Test de la resistance tenken
9,BTCBUSD,1d,1611187200000,1.611274e+12,2021-01-21 00:00:00.0000000,2021-01-21 23:59:59.9990000,35523.1,30844.2,35639.1,30027.0,...,NaN,NA,NA,NaN,Tenken,Prix,Cassure,Support,Sell,cassure du support tenken


In [7]:
test_df = generate_dataset(df_1d, 100)
test_df.head()

C:\Users\benja\AppData\Local\Temp/ipykernel_9700/1567619921.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_output[f"CP_T-{k}"] = df_output["ClosePrice"].shift(k)


,Symbol,Interval,TimeOpenLong,TimeCloseLong,DateOpen,DateClose,OpenPrice,ClosePrice,MaxPrice,MinPrice,...,CP_T-91,CP_T-92,CP_T-93,CP_T-94,CP_T-95,CP_T-96,CP_T-97,CP_T-98,CP_T-99,CP_T-100
0,BTCBUSD,1d,1610409600000,1.610496e+12,2021-01-12 00:00:00.0000000,2021-01-12 23:59:59.9990000,36060.5,34021.1,36615.1,32520.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BTCBUSD,1d,1610496000000,1.610582e+12,2021-01-13 00:00:00.0000000,2021-01-13 23:59:59.9990000,33877.8,37355.7,37763.5,32346.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BTCBUSD,1d,1610582400000,1.610669e+12,2021-01-14 00:00:00.0000000,2021-01-14 23:59:59.9990000,37451.4,39091.6,40132.9,36615.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BTCBUSD,1d,1610668800000,1.610755e+12,2021-01-15 00:00:00.0000000,2021-01-15 23:59:59.9990000,39164.4,36737.6,39703.3,34362.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BTCBUSD,1d,1610755200000,1.610842e+12,2021-01-16 00:00:00.0000000,2021-01-16 23:59:59.9990000,36739.0,35975.0,38149.1,35299.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
len(df_1d)

503

In [28]:
df_1d.to_csv('1d_BTC.csv')

In [17]:
query = extract_data('BTCBUSD', limit=None, interval='1d')
df2 = pd.read_sql_query(query, conn)

In [18]:
df2.head()

,MarketPriceId,Symbol,Interval,TimeOpenLong,TimeCloseLong,DateOpen,DateClose,OpenPrice,ClosePrice,MaxPrice,...,TendancePrice,TendanceLaginSpan,MarketPriceId_laginSpan,MarketPriceId,Reference,Variable,SignalType,LevelType,SignalSens,Comment
0,1122,BTCBUSD,1d,1619654400000,1.619741e+12,2021-04-29 00:00:00.0000000,2021-04-29 23:59:59.9990000,54935.8,53620.9,55266.4,...,Neutre,Baissier,1298.0,1122.0,Cloud,Prix,Entrée par le haut nuage,Ressistance,Sell,Entrée dans le nuage signal baissier
1,1123,BTCBUSD,1d,1618099200000,1.618186e+12,2021-04-11 00:00:00.0000000,2021-04-11 23:59:59.9990000,59860.9,60056.8,60794.5,...,Haussier,Haussier,1400.0,NaN,None,None,None,None,None,None
2,1124,BTCBUSD,1d,1641254400000,1.641341e+12,2022-01-04 00:00:00.0000000,2022-01-04 23:59:59.9990000,46446.0,45823.5,47513.0,...,Baissier,Baissier,37890.0,NaN,None,None,None,None,None,None
3,1125,BTCBUSD,1d,1645401600000,1.645488e+12,2022-02-21 00:00:00.0000000,2022-02-21 23:59:59.9990000,38380.6,37010.5,39498.1,...,Baissier,NA,1268.0,1125.0,Cloud,Prix,Test du bas du nuage,Ressistance,Sell,Test SSA nuage baissier
4,1126,BTCBUSD,1d,1637712000000,1.637798e+12,2021-11-24 00:00:00.0000000,2021-11-24 23:59:59.9990000,57575.9,57170.1,57770.0,...,Neutre,Baissier,1361.0,NaN,None,None,None,None,None,None


503
